# IMDb 文本情感分析

## RNN分析——深度学习初体验（坐牢）

In [1]:
#测试cuda是否可用
import torch
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available.")

CUDA is available.


In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
#导入数据
import pandas as pd
data = pd.read_csv('E:/本科/数据挖掘与商务分析/hw/final/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
#将数据集分为训练集、验证集和测试集
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=1)

In [ ]:
#准备dataset
test_dataset = [(x, y) for x, y in zip(X_test, y_test)]
train_dataset = [(x, y) for x, y in zip(X_train, y_train)]
valid_dataset = [(x, y) for x, y in zip(X_valid, y_valid)]

In [ ]:
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for review, _ in train_dataset:
    tokens = tokenizer(review)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 86341


In [ ]:
#利用torchtext的vocab将token转换为整数
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True) 
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])

[11, 7, 35, 467]


In [ ]:
#定义转换函数
import torch.nn as nn
device = torch.device('cuda' )
review_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
sentiment_pipeline = lambda x: 1 if x == 'positive' else 0

def collate_batch(batch):
    review_list, sentiment_list, lengths = [], [], []
    for _review, _sentiment,in batch:
        processed_text =torch.tensor(review_pipeline(_review), dtype=torch.int64)
        review_list.append(processed_text)
        sentiment_list.append(sentiment_pipeline(_sentiment))
        lengths.append(processed_text.size(0))
    sentiment_list = torch.tensor(sentiment_list)
    lengths = torch.tensor(lengths)
    padded_review_list = nn.utils.rnn.pad_sequence(review_list, batch_first=True)
    return padded_review_list.to(device), sentiment_list.to(device), lengths.to(device)

In [ ]:
#small batch 看看实力
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
review_batch, sentiment_batch, length_batch = next(iter(dataloader))
print(review_batch)
print(sentiment_batch)
print(length_batch)
print(review_batch.shape)

tensor([[   11,     7,    39,  ...,     0,     0,     0],
        [   10,  1304,     6,  ...,     0,     0,     0],
        [    2,   590,  2348,  ..., 25611,   785, 29777],
        [   10,   103,    11,  ...,     0,     0,     0]], device='cuda:0')
tensor([1, 1, 0, 1], device='cuda:0')
tensor([164, 120, 448, 254], device='cuda:0')
torch.Size([4, 448])


In [ ]:
#batch data生成data loader
batch_size = 32 

train_dl = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [ ]:
#将句子转为嵌入编码
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3, padding_idx=0)

#example
text_encoded_input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
print(embedding(text_encoded_input))

tensor([[[ 1.3474, -0.2270,  0.9934],
         [ 1.2762, -1.2231, -1.2136],
         [-1.2001, -0.9781,  0.0818],
         [-1.9540,  0.7038,  0.4304]],

        [[-1.2001, -0.9781,  0.0818],
         [ 0.5202, -0.0546, -0.3535],
         [ 1.2762, -1.2231, -1.2136],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)


### 构建循环神经网络

In [ ]:
#用RNN进行情感分析
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,  batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(86343, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
#编写train函数
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
        lengths = lengths.to(device)
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
            lengths = lengths.to(device)
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
            # 收集所有预测分数和真实标签
            all_labels.extend(label_batch.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset), all_labels, all_preds

In [ ]:
#编写损失函数
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)

#开始显卡炼丹 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid,_,_ = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')


Epoch 0 accuracy: 0.5945 val_accuracy: 0.6593
Epoch 1 accuracy: 0.6984 val_accuracy: 0.7422
Epoch 2 accuracy: 0.7612 val_accuracy: 0.8031
Epoch 3 accuracy: 0.8486 val_accuracy: 0.8464
Epoch 4 accuracy: 0.8827 val_accuracy: 0.8664
Epoch 5 accuracy: 0.9051 val_accuracy: 0.8744
Epoch 6 accuracy: 0.9263 val_accuracy: 0.8784
Epoch 7 accuracy: 0.9401 val_accuracy: 0.8769
Epoch 8 accuracy: 0.9527 val_accuracy: 0.8814
Epoch 9 accuracy: 0.9631 val_accuracy: 0.8855


In [ ]:
#用sklearn评估测试集效果
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
acc_test, loss_test, true_labels, pred_scores= evaluate(test_dl)

# 二分类的阈值设置为0.5
pred_labels = [1 if score >= 0.5 else 0 for score in pred_scores]
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
auc = roc_auc_score(true_labels, pred_scores)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC Score: {auc:.3f}')

Precision: 0.858
Recall: 0.934
F1 Score: 0.894
AUC Score: 0.955


### 双向循环神经网络

In [ ]:
#用RNN进行情感分析
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(86343, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
#编写train函数
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
        lengths = lengths.to(device)
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
            lengths = lengths.to(device)
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
            # 收集所有预测分数和真实标签
            all_labels.extend(label_batch.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset), all_labels, all_preds

In [ ]:
#编写损失函数
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)

#开始显卡炼丹 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid,_,_ = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

Epoch 0 accuracy: 0.6147 val_accuracy: 0.7330
Epoch 1 accuracy: 0.7090 val_accuracy: 0.7538
Epoch 2 accuracy: 0.8048 val_accuracy: 0.8373
Epoch 3 accuracy: 0.8787 val_accuracy: 0.8520
Epoch 4 accuracy: 0.9123 val_accuracy: 0.8761
Epoch 5 accuracy: 0.9342 val_accuracy: 0.8782
Epoch 6 accuracy: 0.9509 val_accuracy: 0.8818
Epoch 7 accuracy: 0.9643 val_accuracy: 0.8534
Epoch 8 accuracy: 0.9765 val_accuracy: 0.8779
Epoch 9 accuracy: 0.9850 val_accuracy: 0.8816


In [ ]:
#用sklearn评估测试集效果
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
acc_test, loss_test, true_labels, pred_scores= evaluate(test_dl)

# 二分类的阈值设置为0.5
pred_labels = [1 if score >= 0.5 else 0 for score in pred_scores]
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
auc = roc_auc_score(true_labels, pred_scores)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC Score: {auc:.3f}')

Precision: 0.901
Recall: 0.865
F1 Score: 0.883
AUC Score: 0.952


### 增加神经网络层数

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn1 = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True, bidirectional=True)
        self.rnn2 = nn.LSTM(rnn_hidden_size*2, rnn_hidden_size, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, fc_hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn1(out)
        out, _ = self.rnn2(out)  # 这里需要使用pad_packed_sequence来恢复序列
        out = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)  # 添加这行代码
        out, output = out  # 现在out是恢复后的序列，output是序列的隐藏状态
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)  # 这里需要确保hidden的形状正确
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(86343, 20, padding_idx=0)
  (rnn1): LSTM(20, 64, batch_first=True, bidirectional=True)
  (rnn2): LSTM(128, 64, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
#编写train函数
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
        lengths = lengths.to(device)
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device).float()  # 确保标签是浮点类型
            lengths = lengths.to(device)
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
            # 收集所有预测分数和真实标签
            all_labels.extend(label_batch.cpu().numpy())
            all_preds.extend(pred.cpu().numpy())
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset), all_labels, all_preds

In [ ]:
#编写损失函数
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)

#开始显卡炼丹 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid,_,_ = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

Epoch 0 accuracy: 0.6402 val_accuracy: 0.6529
Epoch 1 accuracy: 0.7530 val_accuracy: 0.7700
Epoch 2 accuracy: 0.7940 val_accuracy: 0.8285
Epoch 3 accuracy: 0.8718 val_accuracy: 0.8660
Epoch 4 accuracy: 0.9062 val_accuracy: 0.8819
Epoch 5 accuracy: 0.9309 val_accuracy: 0.8862
Epoch 6 accuracy: 0.9469 val_accuracy: 0.8910
Epoch 7 accuracy: 0.9608 val_accuracy: 0.8851
Epoch 8 accuracy: 0.9722 val_accuracy: 0.8825
Epoch 9 accuracy: 0.9821 val_accuracy: 0.8882


In [ ]:
#用sklearn评估测试集效果
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
acc_test, loss_test, true_labels, pred_scores= evaluate(test_dl)

# 二分类的阈值设置为0.5
pred_labels = [1 if score >= 0.5 else 0 for score in pred_scores]
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
auc = roc_auc_score(true_labels, pred_scores)
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC Score: {auc:.3f}')

Precision: 0.906
Recall: 0.881
F1 Score: 0.893
AUC Score: 0.955


可能由于过拟合问题，难以进一步提升

## 引入Transformer——Attention is all you need

In [4]:
torch.backends.cudnn.deterministic = True #确保结果可重现
torch.manual_seed(1)
device = torch.device('cuda' )
num_epochs = 3

#### 运用预训练BERT模型

In [5]:
#使用BERT的tokenizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
# 检查 X_train 的长度 
print(f'Length of X_train: {len(X_train)}')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
valid_encodings = tokenizer(list(X_valid), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)
train_encodings[0]
print(f'Length of train_encodings["input_ids"]: {len(train_encodings["input_ids"])}') 
print(f'Length of valid_encodings["input_ids"]: {len(valid_encodings["input_ids"])}') 
print(f'Length of test_encodings["input_ids"]: {len(test_encodings["input_ids"])}')

Length of X_train: 32000
Length of train_encodings["input_ids"]: 32000
Length of valid_encodings["input_ids"]: 8000
Length of test_encodings["input_ids"]: 10000


In [6]:
#准备dataset
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx],dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

label_map = {'positive': 1, 'negative': 0}

y_train = np.array([label_map[label] for label in y_train]) 
y_valid = np.array([label_map[label] for label in y_valid]) 
y_test = np.array([label_map[label] for label in y_test])

train_dataset = IMDbDataset(train_encodings, y_train)
valid_dataset = IMDbDataset(valid_encodings, y_valid)
test_dataset = IMDbDataset(test_encodings, y_test)

# 打印数据集的长度
print(f'Length of train_dataset: {len(train_dataset)}')
print(f'Length of valid_dataset: {len(valid_dataset)}')
print(f'Length of test_dataset: {len(test_dataset)}')

Length of train_dataset: 32000
Length of valid_dataset: 8000
Length of test_dataset: 10000


In [7]:
#准备dataloader
from torch.utils.data import DataLoader
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_dl =  DataLoader(test_dataset, batch_size=16, shuffle=False)

# 打印数据加载器的长度
print(f'Length of train_dl: {len(train_dl)}')
print(f'Length of valid_dl: {len(valid_dl)}')
print(f'Length of test_dl: {len(test_dl)}')


Length of train_dl: 2000
Length of valid_dl: 500
Length of test_dl: 625


In [8]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.

In [9]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        
        for batch_idx, batch in enumerate(data_loader):
        
        ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
        
        return correct_pred.float()/num_examples

#### 手动微调BERT

In [10]:
import time
start_time = time.time()

for epoch in range(num_epochs):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_dl):
        
        ### Prepare data
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{num_epochs:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dl):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_dl, device):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_dl, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_dl, device):.2f}%')

Epoch: 0001/0003 | Batch 0000/2000 | Loss: 0.6908
Epoch: 0001/0003 | Batch 0250/2000 | Loss: 0.4078
Epoch: 0001/0003 | Batch 0500/2000 | Loss: 0.5256
Epoch: 0001/0003 | Batch 0750/2000 | Loss: 0.2178
Epoch: 0001/0003 | Batch 1000/2000 | Loss: 0.4660
Epoch: 0001/0003 | Batch 1250/2000 | Loss: 0.0117
Epoch: 0001/0003 | Batch 1500/2000 | Loss: 0.2185
Epoch: 0001/0003 | Batch 1750/2000 | Loss: 0.0909
Training accuracy: 0.96%
Valid accuracy: 0.93%
Time elapsed: 16.92 min
Epoch: 0002/0003 | Batch 0000/2000 | Loss: 0.2528
Epoch: 0002/0003 | Batch 0250/2000 | Loss: 0.0874
Epoch: 0002/0003 | Batch 0500/2000 | Loss: 0.0153
Epoch: 0002/0003 | Batch 0750/2000 | Loss: 0.0285
Epoch: 0002/0003 | Batch 1000/2000 | Loss: 0.0619
Epoch: 0002/0003 | Batch 1250/2000 | Loss: 0.1183
Epoch: 0002/0003 | Batch 1500/2000 | Loss: 0.1946
Epoch: 0002/0003 | Batch 1750/2000 | Loss: 0.2867
Training accuracy: 0.99%
Valid accuracy: 0.93%
Time elapsed: 33.79 min
Epoch: 0003/0003 | Batch 0000/2000 | Loss: 0.0876
Epoch: 0

In [11]:
# 在测试集上计算Precision, Recall, F1-score, AUC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

    return all_preds, all_labels

# 获取预测值和真实标签
test_preds, test_labels = evaluate_model(model, test_dl, device)

# 计算各项指标
precision = precision_score(test_labels, test_preds)
recall = recall_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds)
auc = roc_auc_score(test_labels, test_preds)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC: {auc:.3f}')


Precision: 0.919
Recall: 0.943
F1 Score: 0.931
AUC: 0.931


#### 调用自带API微调BERT

In [12]:
#释放显存
del model

In [13]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [14]:
from transformers import Trainer, TrainingArguments

optim = torch.optim.Adam(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3,     
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [15]:
import time
start_time = time.time()
trainer.train()
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

***** Running training *****
  Num examples = 32000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6000
  Number of trainable parameters = 66955010
  0%|          | 10/6000 [00:03<35:52,  2.78it/s]

{'loss': 0.6867, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.01}


  0%|          | 20/6000 [00:07<35:53,  2.78it/s]

{'loss': 0.6225, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}


  0%|          | 30/6000 [00:10<35:52,  2.77it/s]

...

{'loss': 0.1431, 'learning_rate': 8.333333333333334e-08, 'epoch': 3.0}


100%|██████████| 6000/6000 [37:16<00:00,  2.70it/s]Saving model checkpoint to ./results\checkpoint-6000
Configuration saved in ./results\checkpoint-6000\config.json


{'loss': 0.0567, 'learning_rate': 0.0, 'epoch': 3.0}


Model weights saved in ./results\checkpoint-6000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6000/6000 [37:17<00:00,  2.68it/s]

{'train_runtime': 2237.0922, 'train_samples_per_second': 42.913, 'train_steps_per_second': 2.682, 'train_loss': 0.1486751826258066, 'epoch': 3.0}
Total Training Time: 37.29 min


In [16]:
# 在测试集上计算Precision, Recall, F1-score, AUC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

    return all_preds, all_labels

# 获取预测值和真实标签
test_preds, test_labels = evaluate_model(model, test_dl, device)

# 计算各项指标
precision = precision_score(test_labels, test_preds)
recall = recall_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds)
auc = roc_auc_score(test_labels, test_preds)

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC: {auc:.3f}')

Precision: 0.933
Recall: 0.937
F1 Score: 0.935
AUC: 0.935
